# Metagene plots
Now, let's draw **metagene plots**: (using seprate conda env and ipython kernel)

In [1]:
%reload_ext rpy2.ipython

In [3]:
%%R
library(Guitar)

In [4]:
%%R 
txdb <- makeTxDbFromGFF('/rumi/shams/abe/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf',organism='Homo sapiens')

R[write to console]: Import genomic features from the file as a GRanges object ... 
R[write to console]: OK

R[write to console]: Prepare the 'metadata' data frame ... 
R[write to console]: OK

R[write to console]: Make the TxDb object ... 
R[write to console]: OK



In [5]:
%%R 
GuitarPlot(txTxdb = txdb,stBedFiles = list("radar/result.sig.bed"),miscOutFilePrefix = "Guitar")

[1] "20201206021304"
[1] "There are 228048 transcripts of 60669 genes in the genome."
[1] "total 228048 transcripts extracted ..."
[1] "total 97941 transcripts left after ambiguity filter ..."
[1] "total 97941 transcripts left after check chromosome validity ..."
[1] "total 10939 mRNAs left after component length filter ..."
[1] "total 60598 ncRNAs left after ncRNA length filter ..."
[1] "generate components for all tx"
[1] "generate components for mRNA"
[1] "generate components for lncRNA"
[1] "generate chiped transcriptome"
[1] "generate coverage checking ranges for tx"
[1] "generate coverage checking ranges for mrna"
[1] "generate coverage checking ranges for ncrna"
[1] "20201206021840"
[1] "import BED file radar/result.sig.bed"
[1] "sample 10 points for Group1"
[1] "start figure plotting for tx ..."
[1] "start figure plotting for mrna ..."
[1] "start figure plotting for ncrna ..."


In [ ]:
# !for f in *.pdf; do bash ~/Workflows/my_scripts/pdf2png.sh $f; done

In [6]:
mv -v Guitar_* plots/

renamed 'Guitar_mrna_test.pdf' -> 'plots/Guitar_mrna_test.pdf'
renamed 'Guitar_ncrna_test.pdf' -> 'plots/Guitar_ncrna_test.pdf'
renamed 'Guitar_tx_test.pdf' -> 'plots/Guitar_tx_test.pdf'


# FIRE 
https://github.com/goodarzilab/FIRE
### Motif analysis 

In [7]:
cat fire/motifs_of_interest.txt

[AG]GAC
[AGT][AG]AC[ACT]


In [9]:
%%bash 
# get exon sequences for significant hits from RADAR 
bedtools getfasta -name -s -fi /rumi/shams/genomes/hg38/hg38.fa \
-bed radar/result.sig.bed \
-split -fo radar/result.sig.fa

In [33]:
!mkdir -p fire 

In [7]:
%%bash 

# prepare fasta file for FIRE
~/anaconda3/envs/perl/bin/perl ~/prep_seqs_for_teiser_run.pl radar/result.sig.fa fire/peaks

Can't locate Markov.pm in @INC (you may need to install the Markov module) (@INC contains: /flash/bin/FIRE-1.1/SCRIPTS /rumi/shams/abe/perl5/lib/perl5/5.26.2/x86_64-linux-thread-multi /rumi/shams/abe/perl5/lib/perl5/5.26.2 /rumi/shams/abe/perl5/lib/perl5/x86_64-linux-thread-multi /rumi/shams/abe/perl5/lib/perl5 /rumi/shams/abe/anaconda3/envs/perl/lib/site_perl/5.26.2/x86_64-linux-thread-multi /rumi/shams/abe/anaconda3/envs/perl/lib/site_perl/5.26.2 /rumi/shams/abe/anaconda3/envs/perl/lib/5.26.2/x86_64-linux-thread-multi /rumi/shams/abe/anaconda3/envs/perl/lib/5.26.2 .) at /rumi/shams/abe/prep_seqs_for_teiser_run.pl line 5.
BEGIN failed--compilation aborted at /rumi/shams/abe/prep_seqs_for_teiser_run.pl line 5.


CalledProcessError: Command 'b'\n# prepare fasta file for FIRE\n~/anaconda3/envs/perl/bin/perl ~/prep_seqs_for_teiser_run.pl radar/result.sig.fa fire/peaks\n'' returned non-zero exit status 2.

In [6]:
%%bash 
cd fire/
# run FIRE --doskipdiscovery=1
perl $FIREDIR/fire.pl 
    --expfile=peaks_teiser.txt \
    --exptype=discrete 
    --fastafile_rna=peaks_teiser.fa 
    --nodups=1 --dodna=0 --dodnarna=0 
    --species=human 
    --doskipdiscovery=1 
    --motiffile_rna=motifs_of_interest.txt 
    --oribiasonly=0

mv peaks_teiser.txt_FIRE/ motifs_of_interest_FIRE
cd ../

Mon Dec  7 01:04:26 UTC 2020
RNA, Processing peaks_teiser.txt_FIRE/RNA/peaks_teiser.txt
RNA, Step 2: skip seed optimization, use -motiffile_rna option instead.
RNA, Step 3: evaluation of motif significances.
Processing motif [AG]GAC.
MI=0.00492797
Shuffle rank=0
Z-score=44.607
Sigificance category= OK-NO-SEED
Robustness (jn_f=3)=10/10

Processing motif [AGT][AG]AC[ACT].
MI=0.00674459
Shuffle rank=0
Z-score=62.008
Sigificance category= OK-NO-SEED
Robustness (jn_f=3)=10/10

RNA, Step 3.5: creating binary expression profiles.
RNA, Step 4: discovery of distance constraints.
Read 2 motifs.
Read 9374 (genes) x 2 (conditions) matrix.
Read 9374 (genes) x 1 (conditions) matrix.
Reading sequences and building motif profiles ...Done.
Average sequence length is 50.
Quantized E vector into 2 bins
Motif [AG]GAC, present in 3271 genes.
       Average distance: MI=0.0008, rank=1730/10000, Z=0.7684, mbins=3
  Orientation bias {5'): MI=0.0049, rank=0/10000, Z=44.8914
  Orientation bias (3'): MI=0.0007, 

corrupted size vs. prev_size
Aborted (core dumped)
Segmentation fault (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
Can't use 'defined(@array)' (Maybe you should just omit the defined()?) at /flash/bin/FIRE-1.1/SCRIPTS/draw_position_histogram.pl line 134.
Can't locate Hypergeom.pm in @INC (you may need to install the Hypergeom module) (@INC contains: /nvme/bins/FIRE-1.1/SCRIPTS/../modules/lib/perl5/site_perl/5.12.4/darwin-thread-multi-2level/nvme/bins/FIRE-1.1/SCRIPTS/../modules/lib/perl5/site_perl/5.10.0/darwin-thread-multi-2level/Hypergeom.pm /nvme/bins/FIRE-1.1/SCRIPTS/../modules/lib/perl5/site_perl/5.18.2/darwin-thread-multi-2level/Hypergeom.pm /nvme/bins/FIRE-1.1/SCRIPTS/../modules/lib/perl5/site_perl/5.22.0/x86_64-linux/Hypergeom.pm /flash/bin/FIRE-1.1/SCRIPTS /rumi/shams/abe/perl5/lib/perl5/x86_64-linux-gnu-thread-multi /rumi/shams/abe/perl5/lib/perl5 /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.26.1 /usr/local/share/perl/5.26.1 /usr/lib/x86_64-linux-gnu/p

In [38]:
%%bash 
cd fire/
# run FIRE --doskipdiscovery=0
perl $FIREDIR/fire.pl --expfile=peaks_teiser.txt --exptype=discrete --fastafile_rna=peaks_teiser.fa --nodups=1 --dodna=0 --dodnarna=0 --species=human --oribiasonly=0
mv peaks_teiser.txt_FIRE/ discovery_FIRE
cd ../

Thu Aug  6 14:08:07 UTC 2020
RNA, Processing peaks_teiser.txt_FIRE/RNA/peaks_teiser.txt
RNA, Step 1: seed discovery.
Setting -shuffle to 16384.
Expression file is a 9374 by 1 matrix.
Min expression value read is 0, max is 1
Reading sequences ... Done
Number of ORFs = 9374
Calculating Mutual Information for all seeds ... Done
top 10 seeds...
CCCCCCC
Determining threshold ...
Evaluating seed #0, CCCCCCC ... passed.
Evaluating seed #100, AGGTGGA ... not passed.
Decreasing intervals phase.
Evaluating seed #50, GCTAGGC ... not passed.
Evaluating seed #25, TACAAAT ... passed.
Evaluating seed #37, GAGGAAG ... passed.
Evaluating seed #43, GAAACCC ... not passed.
Searches for 5 consecutive 'not passed'.
Threshold set to seed #37 (included).
CCCCCCC	0.0073
GGGGGGG	0.0061
AATGTGG	0.0048
CCCCCCT	0.0043
TCCCCCC	0.0040
GAGAAAC	0.0039
AGGGGGG	0.0039
GCCCCCC	0.0037
AGAAACC	0.0037
TTTTTTT	0.0036
GTGGCAA	0.0034
TTCATAC	0.0033
GAATGTG	0.0031
GGGGGGA	0.0031
CCCCCCA	0.0031
GAGAGAG	0.0029
TCTCTCA	0.0028
TGG

genregexp: malloc.c:2401: sysmalloc: Assertion `(old_top == initial_top (av) && old_size == 0) || ((unsigned long) (old_size) >= MINSIZE && prev_inuse (old_top) && ((unsigned long) old_end & (pagesize - 1)) == 0)' failed.
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
genregexp: malloc.c:2401: sysmalloc: Assertion `(old_top == initial_top (av) && old_size == 0) || ((unsigned long) (old_size) >= MINSIZE && prev_inuse (old_top) && ((unsigned long) old_end & (pagesize - 1)) == 0)' failed.
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
Segmentation fault (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
Segmentation fault (

In [12]:
%%bash 
for f in  fire/*/RNA/*.pdf; do 
    o=${f/\/RNA\/peaks_teiser.txt./_}
    mv -v $f $o; 
done 

renamed 'fire/discovery_FIRE/RNA/peaks_teiser.txt.densities.pdf' -> 'fire/discovery_FIRE_densities.pdf'
renamed 'fire/discovery_FIRE/RNA/peaks_teiser.txt.fullmimatrix.pdf' -> 'fire/discovery_FIRE_fullmimatrix.pdf'
renamed 'fire/discovery_FIRE/RNA/peaks_teiser.txt.summary.pdf' -> 'fire/discovery_FIRE_summary.pdf'
renamed 'fire/motifs_of_interest_FIRE/RNA/peaks_teiser.txt.densities.pdf' -> 'fire/motifs_of_interest_FIRE_densities.pdf'
renamed 'fire/motifs_of_interest_FIRE/RNA/peaks_teiser.txt.fullmimatrix.pdf' -> 'fire/motifs_of_interest_FIRE_fullmimatrix.pdf'
renamed 'fire/motifs_of_interest_FIRE/RNA/peaks_teiser.txt.summary.pdf' -> 'fire/motifs_of_interest_FIRE_summary.pdf'


In [2]:
!for f in fire/*pdf; do bash /rumi/shams/abe/Workflows/my_scripts/pdf2png.sh $f; done

fire/discovery_FIRE_densities.pdf > fire/discovery_FIRE_densities.png
done!
fire/discovery_FIRE_fullmimatrix.pdf > fire/discovery_FIRE_fullmimatrix.png
done!
fire/discovery_FIRE_summary.pdf > fire/discovery_FIRE_summary.png
done!
fire/motifs_of_interest_FIRE_densities.pdf > fire/motifs_of_interest_FIRE_densities.png
done!
fire/motifs_of_interest_FIRE_fullmimatrix.pdf > fire/motifs_of_interest_FIRE_fullmimatrix.png
done!
fire/motifs_of_interest_FIRE_summary.pdf > fire/motifs_of_interest_FIRE_summary.png
done!


### DRACH & RGAC motifs

<img src=fire/motifs_of_interest.png style="width:1100px">

## Discover over/under represented motifs 

<img src=fire/discovery_FIRE.png style="width:1100px">

# iPAGE
https://github.com/goodarzilab/PAGE

In [15]:
!nohup bash ipage_human_ensembl.sh d_mtyl_T_vs_U.txt > ipage_human_ensembl.out 

nohup: ignoring input and redirecting stderr to stdout


<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_h.png" title="msigdb_c1" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_c1.png" title="msigdb_c1" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_c2.png" title="msigdb_c2" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_c3.png" title="msigdb_c3" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_c4.png" title="msigdb_c4" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_c5.png" title="msigdb_c5" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_c6.png" title="msigdb_c6" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_c7.png" title="msigdb_c7" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_msigdb_full.png" title="msigdb_full" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl.png" title="ensembl" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_RBPs_coding_gene_ids_by_5UTR.png" title="RBPs_coding_gene_ids_by_5UTR" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_RBPs_coding_gene_ids_by_coding_exons.png" title="RBPs_coding_gene_ids_by_coding_exons" style="width:600px">
<img src="plots/d_mtyl_T_vs_U.ipage.human_ensembl_RBPs_coding_gene_ids_by_introns.png" title="RBPs_coding_gene_ids_by_introns" style="width:600px">


In [16]:
%%R
sessionInfo()

UsageError: Cell magic `%%R` not found.
